# Sugar Disease Classification

## Kaggle APIs

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 73 bytes


## Dowload Dataset

In [ ]:
!kaggle competitions download -c durian-hackathon-fruit-classification

 97% 55.0M/56.8M [00:03<00:00, 24.9MB/s]
100% 56.8M/56.8M [00:03<00:00, 16.8MB/s]


In [ ]:
!unzip /content/durian-hackathon-fruit-classification.zip

Archive:  /content/durian-hackathon-fruit-classification.zip
  inflating: Images/001c68734a6f348e981906082144b91f.jpg  
  inflating: Images/0021402bd597fa24845b7a5d4a753a34.jpg  
  inflating: Images/00292b8cf7cbab755e657dca7d583bbc.jpg  
  inflating: Images/0031ef84d762c39229609b4717c0ba55.jpg  
  inflating: Images/003700f1a459758acad179d50d3186cf.jpg  
  inflating: Images/00557145357cedb92d4a4c083ef3b730.jpg  
  inflating: Images/006e39b0b47cc8a17a0e94cec263b460.jpg  
  inflating: Images/0071492300bdbd6b13b6daad13420c47.jpg  
  inflating: Images/007400a8e581f9c9f3fe91d392156c14.jpg  
  inflating: Images/009122462215c2775ee113f70f023cf8.jpg  
  inflating: Images/00aa896a573318097c90a7aa5e8674c0.jpg  
  inflating: Images/00e0142403a637fb534056a7640cfdd6.jpg  
  inflating: Images/00e16ba564b1cfd4d3b27f0a4252a0d2.jpg  
  inflating: Images/00efd7d3b4f256e0d22bfa38c99b9165.jpg  
  inflating: Images/00f619a334d95b568dc34d786ed2dbe0.jpg  
  inflating: Images/00f7a3c3ddc4e67a81b99738472f3bfe.j

# Perform CLIP model

In [ ]:
from transformers import pipeline
from PIL import Image
import pandas as pd

# Load model from checkpoint
checkpoint = "zer0int/CLIP-GmP-ViT-L-14"
detector = pipeline(model=checkpoint, task="zero-shot-image-classification")

# Read test csv
test_csv = pd.read_csv("./test.csv")
test_csv.head()

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/902 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Device set to use cuda:0


,id
0,5041188f151a9140d914ec63c7f0cd24.jpg
1,ea064354a65041c10f10d8989dd98a0b.jpg
2,b5353684e442dfe99ae5fd64d1179dca.jpg
3,5d71aeb9bff60de90d03500ca5fdd03b.jpg
4,69116bf0f93e7277fbed94f67c1c4b18.jpg


In [ ]:
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np
from tqdm import tqdm

candidate_labels = ["Avocado", "Cocoa Bean", "Coconut", "Durian", "Eggplant", "Fig", "Guava"]
labels = []

for image_path in tqdm(test_csv['id'].to_list()):
    image_path = "Images/" + image_path
    image = Image.open(image_path).convert("RGB")

    # Predict the label
    predictions = detector(image, candidate_labels=candidate_labels)
    labels.append(predictions[0]['label'])

# print(labels)

100%|██████████| 1052/1052 [01:21<00:00, 12.97it/s]


In [ ]:
candidate_labels = ["Avocado", "Cocoa Bean", "Coconut", "Durian", "Eggplant", "Fig", "Guava"]
mlabels = []

for label in labels:
  mlabels.append(candidate_labels.index(label))

In [ ]:
submission = pd.DataFrame({
    "id": test_csv['id'],
    "predict": mlabels
})

submission.to_csv("submission.csv", index=False)

In [ ]:
!kaggle competitions submit -c durian-hackathon-fruit-classification -f /content/submission.csv -m "CLIP patch14"

100% 40.1k/40.1k [00:01<00:00, 23.8kB/s]
Successfully submitted to Durian Hackathon: Fruit Classification

In [ ]:
!kaggle competitions submissions -c durian-hackathon-fruit-classification

fileName         date                 description             status    publicScore  privateScore  
---------------  -------------------  ----------------------  --------  -----------  ------------  
submission.csv   2025-02-15 08:15:43  CLIP patch14            complete  0.9752                     
submission.csv   2025-02-15 08:11:53  Resnet-101              complete  0.9694                     
submission.csv   2025-02-15 08:11:09  CLIP patch14            complete  0.2071                     
submit_ture.csv  2025-02-15 08:09:28  Clip patch32 inference  complete  0.8166                     
submit_ture.csv  2025-02-15 08:07:19  Clip patch32 inference  error                                
submission.csv   2025-02-15 08:06:38  CLIP patch14            complete  0.9886                     
submission.csv   2025-02-15 08:00:52  CLIP patch14            complete  0.8982                     
submission.csv   2025-02-15 07:58:53  CLIP patch14            complete  0.8731                     
